# Notebook for MST Analysis

In [1]:
#importing libraries
import utils as u
import numpy as np
import pandas as pd
import pynndescent
from scipy.sparse.csgraph import connected_components
import matplotlib.pyplot as plt

In [2]:
#loading datasets
dsname = "Datasets/blobs_size_2000_centers_10_outliers_20.csv"
dataset = pd.read_csv(dsname, header=None).values
#defining k
k = 20
mpts = k 

In [3]:
#Run pyNNDescent
index = pynndescent.NNDescent(dataset, n_neighbors = k, tree_init = False, verbose=True)
#Convert pyNNDescent to Igor's Structure
knndescent = u.convert_pynndescent_output(index.neighbor_graph)
#building knn + residual information
knnRdescent = index.residual
knnRdescent = [nn[1:]for nn in knnRdescent]
#Calculate and build exact 
sorted_dist_m = u.get_sorted_dist_matrix(dataset)

Tue Sep  7 14:41:10 2021 NN descent for 11 iterations
[LOW] 	 1  /  11
[LOW] 	 2  /  11
[LOW] 	 3  /  11
[LOW] 	 4  /  11
	Stopping threshold met -- exiting after 4 iterations


In [4]:
#definition of method for MST analisys
def mst_analysis(dataset, approx_knn, approx_knn_residual, sorted_distance_matrix, k=1, plot=True):
    
 
    #calculate the MSF of the kkn of the nndescennt
    knn_mst = u.get_mst(u.to_adj_m(approx_knn))
    
    #calculate the MSF of the complete graph using the knn of the nndescennt
    knn_fmst = u.f_boruvka(approx_knn, k)
    
    #calculate the MSF of the kkn of the nndescennt + residual
    knnR_mst = u.get_mst(u.to_adj_m(approx_knn_residual))
    
    #calculate the MSF of the complete graph using the knn of the nndescennt + residual
    knnR_fmst = u.f_boruvka(approx_knn_residual, k)
    
    #find true knn and real complete MST
    #trueknn = u.get_knn(sorted_distance_matrix, k)   
    real_mst = u.get_mst(u.to_adj_m(sorted_distance_matrix))
    
    #extract edges 
    knn_mst_edges = set(u.get_edges(knn_mst.toarray()))
    knn_fmst_edges = set(u.get_edges(knn_fmst))
    knnR_mst_edges = set(u.get_edges(knnR_mst.toarray()))
    knnR_fmst_edges = set(u.get_edges(knnR_fmst))   
    real_mst_edges = set(u.get_edges(real_mst.toarray()))
    
    if plot:
        ax = plt.subplot(2,3,1)
        ax.set_aspect('equal')
        u.plot_edges(dataset, knn_mst.toarray())
        u.plot_dataset(dataset)
        plt.title(f"KNN K={k} MST")
        
        ax = plt.subplot(2,3,2)
        ax.set_aspect('equal')
        u.plot_edges(dataset, knn_fmst)
        u.plot_dataset(dataset)
        plt.title(f"KNN K={k} MST-Frozen")

        ax = plt.subplot(2,3,3)
        ax.set_aspect('equal')
        u.plot_edges(dataset, knnR_mst.toarray())
        u.plot_dataset(dataset)
        plt.title(f"KNN + Residual K={k} MST")
        
        ax = plt.subplot(2,3,4)
        ax.set_aspect('equal')
        u.plot_edges(dataset, knnR_fmst)
        u.plot_dataset(dataset)
        plt.title(f"KNN + Residual K={k} MST-Frozen")
        
        ax = plt.subplot(2,3,5)
        ax.set_aspect('equal')
        u.plot_edges(dataset, real_mst.toarray())
        u.plot_dataset(dataset)
        plt.title(f"MST")
    
    return (knn_mst_edges, knn_fmst_edges, knnR_mst_edges, knnR_fmst_edges, real_mst_edges)

In [5]:
#plotting knn
plt.figure(figsize=(16,12))
knn_edges, knnf_edges, knnR_edges, knnRf_edges, real_edges = mst_analysis(dataset, knndescent, knnRdescent, sorted_dist_m, k, plot=True)
plt.show()

AttributeError: module 'utils' has no attribute 'f_boruvka'

<Figure size 1152x864 with 0 Axes>

In [ ]:
#printing MST analysis
print(f"k = {k}")
print(f"Number of KNN MSF edges that are not in the MST = {len(knn_edges - real_edges)}/{len(knn_edges)}")
print(f"Number of KNN Frozen MSF edges that are not in the MST = {len(knnf_edges - real_edges)}/{len(knnf_edges)}")
print(f"Number of KNN+Residual MSF edges that are not in the MST = {len(knnR_edges - real_edges)}/{len(knnR_edges)}")
print(f"Number of KNN+Residual Frozen MSF edges that are not in the MST = {len(knnRf_edges - real_edges)}/{len(knnRf_edges)}")
print(f"MST no edges: {len(real_edges)}")
print("---------------------------------------------------\n")

# MST over densities estimations


In [ ]:
#function that calculates mutual reachability distances
def calculate_mrd(sorted_distance_matrix, mpts):

    #calculating core distances
    core_dist = [sorted_distance_matrix[i][mpts-1][0] for i in range(len(sorted_distance_matrix))]
    
    #calculating mrd wrt mpts    
    for i in range(len(sorted_distance_matrix)):
        for j in range(len(sorted_distance_matrix[i])):
            sorted_distance_matrix[i][j] = (max(sorted_distance_matrix[i][j][0],core_dist[i], core_dist[j]),sorted_distance_matrix[i][j][1])
        sorted_distance_matrix[i].sort()

    return (sorted_distance_matrix, core_dist)

In [ ]:
#calculate mrd and core true distances
mpts = k
mrd_dist , core_dist = calculate_mrd(sorted_dist_m, mpts)

#calculate approximatted mrd and core distances
approx_mrd_dist , approx_core_dist = calculate_mrd(knnRdescent, mpts)

#extracting approximatted  knn
approximatedknn = u.get_knn(approx_mrd_dist, k) 

#extracting knn
trueknn = u.get_knn(mrd_dist, k) 


In [ ]:
plt.figure(figsize=(16,12))
knn_edges, knnf_edges, knnR_edges, knnRf_edges, real_edges = mst_analysis(dataset, approximatedknn, approx_mrd_dist, mrd_dist, k, plot=True)
plt.show()

In [ ]:
#printing MST analysis
print(f"k = {k}")
print(f"Number of KNN MSF edges that are not in the MST = {len(knn_edges - real_edges)}/{len(knn_edges)}")
print(f"Number of KNN Frozen MSF edges that are not in the MST = {len(knnf_edges - real_edges)}/{len(knnf_edges)}")
print(f"Number of KNN+Residual MSF edges that are not in the MST = {len(knnR_edges - real_edges)}/{len(knnR_edges)}")
print(f"Number of KNN+Residual Frozen MSF edges that are not in the MST = {len(knnRf_edges - real_edges)}/{len(knnRf_edges)}")
print(f"MST no edges: {len(real_edges)}")
print("---------------------------------------------------\n")


## Setup and Install

In [ ]:
# Install a pip package in the current Jupyter kernel
#import sys
#!{sys.executable} -m pip install -e ~/notebook/pynndescent-master/
#!{sys.executable} -m pip install pynndescent
#!{sys.executable} -m pip uninstall -y pynndescent
